In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [11]:
dataset = 'onion' # onion or emma
results_dir = f'../results/' + dataset + '/performance/'
cutoffs = [1, 5, 10, 20]
metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR', 'EFD', 'EPC', 'ARP', 'PopREO', 'PopRSP', 'ItemCoverage', 'UserCoverage']
#metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR']

In [12]:
def find_modality(string):
    keyword = "modalites="
    start = string.find(keyword)
    if start == -1:
        return None

    start += len(keyword)
    end = string.find("_", start)
    if end == -1:
        end = len(string)

    modality = string[start:end].strip("'")
    return modality

In [13]:
# read all files in result folder
files = os.listdir(results_dir)
df = pd.DataFrame()

for cutoff in cutoffs:
    df_cut = pd.DataFrame()
    result_files_cutoff = [f for f in files if f.startswith(f'rec_cutoff_{cutoff}_')]
    for f in result_files_cutoff:
        df_tmp = pd.read_csv(results_dir + f, sep='\t')
        tmp_metrics = [m for m in metrics if m in df_tmp.columns]
        df_tmp.rename(columns={m: f'{m}@{cutoff}' for m in tmp_metrics}, inplace=True)
        df_tmp['modality'] = find_modality(df_tmp['model'].values[0])
        df_tmp['model'] = df_tmp['model'].str.split('_').str[0]
        df_tmp['date'] = f[-23:-4]
        df_cut = pd.concat([df_cut, df_tmp], axis=0)

    # keep only newest model results
    df_cut = df_cut.sort_values('date').drop_duplicates(subset=['model', 'modality'], keep='last')

    # merge with previous cutoffs
    if not df.empty:
        df = pd.merge(df, df_cut, on=['model', 'date', 'modality'], how='inner')
        print(df.shape)
    else:
        df = df_cut

df = df[['model', 'modality', 'date'] + [c for c in df.columns if '@' in c]]
df.sort_values('nDCG@5', inplace=True)
# display columns with @10 only
#df[['model', 'modality', 'date'] + [c for c in df.columns if '@1' in c]]
df[['model', 'modality', 'date'] + ['Recall@1', 'Recall@5', 'Recall@10', 'nDCG@1', 'nDCG@5', 'nDCG@10']]

(7, 13)
(7, 18)
(7, 23)


,model,modality,date,Recall@1,Recall@5,Recall@10,nDCG@1,nDCG@5,nDCG@10
2,Random,None,2025_02_19_12_23_06,0.000057,0.000142,0.000435,0.000305,0.000249,0.000353
1,MostPop,None,2025_02_19_12_23_06,0.002608,0.009729,0.015334,0.011415,0.010891,0.012566
6,LightGCN,None,2025_02_22_17_47_27,0.044801,0.148772,0.207801,0.160633,0.161059,0.178209
3,FeatureItemKNN,emotion,2025_02_20_03_09_50,0.061262,0.210941,0.293501,0.251863,0.245303,0.264353
0,ItemKNN,None,2025_02_19_12_23_06,0.061633,0.211229,0.293755,0.252908,0.245702,0.264729
5,FeatureItemKNN,textual,2025_02_20_15_39_12,0.061394,0.211460,0.293683,0.252124,0.245749,0.264581
4,FeatureItemKNN,audio,2025_02_20_09_52_00,0.061740,0.211500,0.293630,0.252995,0.245965,0.264736


In [13]:
# export to comma seperated file
df.to_csv(f'../results/{dataset}_performance.csv', index=False)

In [14]:
for cutoff in cutoffs:
    df_cut = df[['model'] + [c for c in df.columns if f'@{cutoff}' in c]]
    df_cut.to_csv(f'../results/{dataset}_performance_{cutoff}.csv', index=False)